# 3rd Question

### Which player has the highest number of wins?

Installing needed packages

In [24]:
%pip install numpy pandas matplotlib pyarrow

Note: you may need to restart the kernel to use updated packages.


Importing needed packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

# Reading .parquet data
According to question and ERD in /data/ERD folder as Tennis Schema.pdf , We are analyzing which player has the highest win rate, Reading data of matchs
It might take few seconds to read data

In [ ]:
days_to_query = os.listdir("../data/raw/tennis_data")
files_to_load = []

for day in days_to_query:
    files_to_load.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_statistics*/statistics_*.parquet"
        )
    )

tennis_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "period",
                "statistic_name",
                "home_stat",
                "away_stat",
                "home_total",
                "away_total",
            ],
        )
        for file in files_to_load
    ],
    ignore_index=True,
)


,match_id,period,statistic_name,home_stat,away_stat,home_total,away_total
0,11998445,ALL,aces,12,6,NaN,NaN
1,11998445,ALL,double_faults,2,7,NaN,NaN
2,11998445,ALL,first_serve,57/101 (56%),53/90 (59%),101.0,90.0
3,11998445,ALL,second_serve,42/44 (95%),30/37 (81%),44.0,37.0
4,11998445,ALL,first_serve_points,42/57 (74%),39/53 (74%),57.0,53.0
...,...,...,...,...,...,...,...
95,11998446,1ST,first_serve_points,12/18 (67%),12/16 (75%),18.0,16.0
96,11998446,1ST,second_serve_points,7/11 (64%),9/12 (75%),11.0,12.0
97,11998446,1ST,service_games_played,5,5,NaN,NaN
98,11998446,1ST,break_points_saved,0/1 (0%),2/2 (100%),1.0,2.0


### Reading data of Matches and Teams

In [40]:
days_to_query_ = os.listdir("../data/raw/tennis_data")
files_to_load_ = []

for day in days_to_query_:
    files_to_load_.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_match*/home_team_1*.parquet"
        )
    )

home_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "name"
            ],
        )
        for file in files_to_load_
    ],
    ignore_index=True,
)

In [42]:
days_to_query__ = os.listdir("../data/raw/tennis_data")
files_to_load__ = []

for day in days_to_query__:
    files_to_load__.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_match*/away_team_1*.parquet"
        )
    )

away_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "name"
            ],
        )
        for file in files_to_load__
    ],
    ignore_index=True,
)


Display data of home matches required information :

In [44]:
home_statistics_data_frame.rename(columns={"name" : "Home Player", "match_id" : "Match_id"}, inplace=True)
home_statistics_data_frame.head()

,Match_id,Home Player
0,11998445,Cazaux A.
1,11998446,Lestienne C.
2,11998447,Ćorić B.
3,11998448,Mmoh M.
4,11998449,Paire B.


Display data of away matches required information :

In [46]:
away_statistics_data_frame.rename(columns={"name" : "Away Player", "match_id" : "Match_id"}, inplace=True)
away_statistics_data_frame.head()

,Match_id,Away Player
0,11998445,Auger-Aliassime F.
1,11998446,Cobolli F.
2,11998447,Martínez P.
3,11998448,Muller A.
4,11998449,Mayot H.


Merging whole data

In [57]:
merged_df = pd.merge(home_statistics_data_frame, away_statistics_data_frame, on="Match_id", how="inner")
merged_df.rename(columns={'Home_Player_Name': 'Home_Player', 'Away_Player_Name': 'Away_Player'}, inplace=True)
merged_df_unique = merged_df.drop_duplicates()
merged_df_unique.head(25)

,Match_id,Home Player,Away Player
0,11998445,Cazaux A.,Auger-Aliassime F.
2,11998446,Lestienne C.,Cobolli F.
4,11998447,Ćorić B.,Martínez P.
6,11998448,Mmoh M.,Muller A.
7,11998449,Paire B.,Mayot H.
8,11998450,Shapovalov D.,Bublik A.
9,11998451,Shevchenko A.,Barrere G.
10,11998456,Shevchenko A.,Bublik A.
13,11998459,Rune H.,Llamas Ruiz P.
15,11998666,Saville L.,Imai S.
